In [1]:
import sys
!{sys.executable} -m pip install tabulate

In [2]:
#nessicary packages
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
import time
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from tabulate import tabulate
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from graphviz import Digraph
import graphviz

In [3]:
#get sample data
file = "trip39A.txt"

df = pd.read_csv(file, delimiter = ",")

In [4]:
#drop redundant data
redudantRows = ["DATASOURCE","BASIN","TENDERLOT","SUPPRESSED","JUSTIFICATIONID","LASTUPDATE","NOTE","Unnamed: 0"]

df.drop(columns=redudantRows, inplace=True)
df.shape

(149, 9)

In [5]:
#test target features
df["PLANNEDTIME_TOTAL"] = df["PLANNEDTIME_ARR"] - df["PLANNEDTIME_DEP"]
df["ACTUALTIME_TOTAL"] = df["ACTUALTIME_ARR"] - df["ACTUALTIME_DEP"]
df["DELAY"] = df["ACTUALTIME_TOTAL"] - df["PLANNEDTIME_TOTAL"]

df.shape

(149, 12)

In [6]:
#Drop NA Values
df.dropna(inplace = True)
df.shape

(127, 12)

In [7]:
#format date column correctly
df['DATE'] = np.nan
for i in range(df.shape[0]):
    df['DATE'].iloc[i] = datetime.strptime(df['DAYOFSERVICE'].iloc[i], '%d-%b-%y 00:00:00')
    
#get hour and minutes
df["hours"] = df["ACTUALTIME_DEP"]//3600
df["minutes"] = ((df["ACTUALTIME_DEP"]/3600 - df["ACTUALTIME_DEP"]//3600)*60)//1
df.dtypes

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


DAYOFSERVICE          object
TRIPID                 int64
LINEID                object
ROUTEID               object
DIRECTION              int64
PLANNEDTIME_ARR        int64
PLANNEDTIME_DEP        int64
ACTUALTIME_ARR       float64
ACTUALTIME_DEP       float64
PLANNEDTIME_TOTAL      int64
ACTUALTIME_TOTAL     float64
DELAY                float64
DATE                  object
hours                float64
minutes              float64
dtype: object

In [8]:
df

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,PLANNEDTIME_TOTAL,ACTUALTIME_TOTAL,DELAY,DATE,hours,minutes
0,07-FEB-18 00:00:00,6246028,39A,39A_40,1,49276,44400,50038.0,44382.0,4876,5656.0,780.0,2018-02-07 00:00:00,12.0,19.0
1,07-FEB-18 00:00:00,6261203,39A,39A_43,2,39535,34800,40249.0,34984.0,4735,5265.0,530.0,2018-02-07 00:00:00,9.0,43.0
2,07-FEB-18 00:00:00,6251013,39A,39A_40,1,71287,66600,72534.0,66602.0,4687,5932.0,1245.0,2018-02-07 00:00:00,18.0,30.0
3,26-JUN-18 00:00:00,7104333,39A,39A_43,2,40074,35400,40759.0,35333.0,4674,5426.0,752.0,2018-06-26 00:00:00,9.0,48.0
4,26-JUN-18 00:00:00,7104336,39A,39A_40,1,58316,53400,58367.0,53393.0,4916,4974.0,58.0,2018-06-26 00:00:00,14.0,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,18-FEB-18 00:00:00,6264923,39A,39A_43,2,46163,42000,46657.0,42023.0,4163,4634.0,471.0,2018-02-18 00:00:00,11.0,40.0
145,18-FEB-18 00:00:00,6274917,39A,39A_43,2,42863,38700,43234.0,38743.0,4163,4491.0,328.0,2018-02-18 00:00:00,10.0,45.0
146,18-FEB-18 00:00:00,6271772,39A,39A_43,2,41063,36900,41036.0,36945.0,4163,4091.0,-72.0,2018-02-18 00:00:00,10.0,15.0
147,18-FEB-18 00:00:00,6263988,39A,39A_43,2,44663,40500,44979.0,40500.0,4163,4479.0,316.0,2018-02-18 00:00:00,11.0,15.0


In [9]:
#Get weather data
dfWeather = pd.read_csv("2018_historic_weather_3.txt", delimiter = "\t",  parse_dates=False)
dfWeather.dt = dfWeather.dt.astype('datetime64[ns]')
#dfWeather.dt

dfWeather.dtypes

dt                     datetime64[ns]
dt_iso                         object
timezone                        int64
city_name                      object
lat                           float64
lon                           float64
temp                          float64
feels_like                    float64
temp_min                      float64
temp_max                      float64
pressure                        int64
sea_level                     float64
grnd_level                    float64
humidity                        int64
wind_speed                    float64
wind_deg                        int64
rain_1h                       float64
rain_3h                       float64
snow_1h                       float64
snow_3h                       float64
clouds_all                      int64
weather_id                      int64
weather_main                   object
weather_description            object
weather_icon                   object
dtype: object

In [10]:
#create propper weather columns
dfWeather['dates'] = np.nan

for i in range(8800):
    dfWeather['dates'].iloc[i] = datetime.strptime(dfWeather['dt_iso'].iloc[i], '%Y-%m-%d %H:%M:%S +0000 UTC')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [11]:
#select features that may have proise with tarket outcome
dfWeather["DATE"] = dfWeather['dates']
dfweather1 = dfWeather[["DATE","temp","feels_like","temp_min","temp_max","pressure","humidity","weather_main"]]

In [12]:
dfweather1.DATE = dfweather1.DATE.astype('datetime64[ns]')
dfweather1.dtypes

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


DATE            datetime64[ns]
temp                   float64
feels_like             float64
temp_min               float64
temp_max               float64
pressure                 int64
humidity                 int64
weather_main            object
dtype: object

In [13]:
#create cleaned wether data set
dfWeather_main = pd.get_dummies(dfweather1['weather_main'])
dfWeather_cleaned = pd.concat([dfweather1[["DATE","temp","feels_like","temp_min","temp_max","pressure","humidity"]],pd.get_dummies(dfweather1['weather_main'])],axis=1)

dfWeather_cleaned["DATES"] = dfWeather_cleaned["DATE"]
dfWeather_cleaned.DATES = dfWeather_cleaned.DATES.astype('datetime64[ns]')
dfWeather_cleaned.dtypes

DATE          datetime64[ns]
temp                 float64
feels_like           float64
temp_min             float64
temp_max             float64
pressure               int64
humidity               int64
Clear                  uint8
Clouds                 uint8
Drizzle                uint8
Fog                    uint8
Mist                   uint8
Rain                   uint8
Smoke                  uint8
Snow                   uint8
DATES         datetime64[ns]
dtype: object

In [14]:
#create datetime column with all the data
df["month"] = np.nan

for i in range(df.shape[0]):
        df["DATE"].iloc[i]= df["DATE"].iloc[i].replace(hour=int(df["hours"].iloc[i]), minute=int(df["minutes"].iloc[i]))
        df["month"].iloc[i] = df.DATE.iloc[i].month

df["month"] = df["month"].astype("int64")        
df

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,PLANNEDTIME_TOTAL,ACTUALTIME_TOTAL,DELAY,DATE,hours,minutes,month
0,07-FEB-18 00:00:00,6246028,39A,39A_40,1,49276,44400,50038.0,44382.0,4876,5656.0,780.0,2018-02-07 12:19:00,12.0,19.0,2
1,07-FEB-18 00:00:00,6261203,39A,39A_43,2,39535,34800,40249.0,34984.0,4735,5265.0,530.0,2018-02-07 09:43:00,9.0,43.0,2
2,07-FEB-18 00:00:00,6251013,39A,39A_40,1,71287,66600,72534.0,66602.0,4687,5932.0,1245.0,2018-02-07 18:30:00,18.0,30.0,2
3,26-JUN-18 00:00:00,7104333,39A,39A_43,2,40074,35400,40759.0,35333.0,4674,5426.0,752.0,2018-06-26 09:48:00,9.0,48.0,6
4,26-JUN-18 00:00:00,7104336,39A,39A_40,1,58316,53400,58367.0,53393.0,4916,4974.0,58.0,2018-06-26 14:49:00,14.0,49.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,18-FEB-18 00:00:00,6264923,39A,39A_43,2,46163,42000,46657.0,42023.0,4163,4634.0,471.0,2018-02-18 11:40:00,11.0,40.0,2
145,18-FEB-18 00:00:00,6274917,39A,39A_43,2,42863,38700,43234.0,38743.0,4163,4491.0,328.0,2018-02-18 10:45:00,10.0,45.0,2
146,18-FEB-18 00:00:00,6271772,39A,39A_43,2,41063,36900,41036.0,36945.0,4163,4091.0,-72.0,2018-02-18 10:15:00,10.0,15.0,2
147,18-FEB-18 00:00:00,6263988,39A,39A_43,2,44663,40500,44979.0,40500.0,4163,4479.0,316.0,2018-02-18 11:15:00,11.0,15.0,2


In [15]:
#df.DATES.astype('object')
df_final = df[["TRIPID","DATE","DIRECTION","PLANNEDTIME_TOTAL","ACTUALTIME_TOTAL","DELAY","LINEID","ROUTEID","hours","month"]]
df_final.DATE = df_final.DATE.astype('datetime64[ns]')
df_final.dtypes

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


TRIPID                        int64
DATE                 datetime64[ns]
DIRECTION                     int64
PLANNEDTIME_TOTAL             int64
ACTUALTIME_TOTAL            float64
DELAY                       float64
LINEID                       object
ROUTEID                      object
hours                       float64
month                         int64
dtype: object

In [16]:
df_total = pd.merge_asof(df_final.sort_values("DATE"), dfWeather_cleaned, left_on="DATE", right_on="DATE")

In [17]:
#convert directions to binary column
df_total.DIRECTION.replace(to_replace=2, value=0, inplace=True)

In [18]:
df_total.corr()[["DELAY","ACTUALTIME_TOTAL"]].sort_values(by=['DELAY'], ascending=False)

,DELAY,ACTUALTIME_TOTAL
DELAY,1.000000,0.700393
ACTUALTIME_TOTAL,0.700393,1.000000
DIRECTION,0.263680,0.227464
pressure,0.122984,0.117790
Mist,0.111460,-0.001347
hours,0.051162,-0.151248
Drizzle,0.049660,-0.089720
PLANNEDTIME_TOTAL,0.034620,0.737577
Clouds,-0.002171,0.262591
temp,-0.043957,0.078778


In [19]:
#Direction, 1 = inbound , 0 = outbound

# What Effects Delay:
- Direction
- Pressure
- Mist
- Rain
- Humidity


# What effects Actualtime_Total:
- Direction
- Pressure
- Hours
- PlannedTime
- Clouds
- Rain
- Humidity
- Month

TRIPID was not included in this as it is a unique identifier for each trip, and there should not be any correlation between the unique identifier for the dataset and the target outcome. 

In [20]:
features = ["DIRECTION","pressure","hours","PLANNEDTIME_TOTAL","Clouds","Rain","humidity","month"]

features = ["DIRECTION","pressure","Mist","Rain","humidity"]

In [21]:
df_total[features]

,DIRECTION,pressure,Mist,Rain,humidity
0,0,1024,0,0,93
1,1,1023,0,0,87
2,1,1019,0,0,87
3,0,1017,0,0,93
4,0,1017,0,1,93
...,...,...,...,...,...
122,0,993,0,0,82
123,0,993,0,0,82
124,0,993,0,0,82
125,0,998,0,0,82


In [22]:
df_total["ACTUALTIME_TOTAL"]

0      5265.0
1      5656.0
2      5932.0
3      3920.0
4      4091.0
        ...  
122    5557.0
123    5587.0
124    5419.0
125    6079.0
126    5755.0
Name: ACTUALTIME_TOTAL, Length: 127, dtype: float64

In [23]:
df_total.dtypes

for i in df_total.columns:
    if df_total[i].dtype == "uint8":
        df_total[i] = df_total[i].astype("int64")

In [24]:
df_total.dtypes

TRIPID                        int64
DATE                 datetime64[ns]
DIRECTION                     int64
PLANNEDTIME_TOTAL             int64
ACTUALTIME_TOTAL            float64
DELAY                       float64
LINEID                       object
ROUTEID                      object
hours                       float64
month                         int64
temp                        float64
feels_like                  float64
temp_min                    float64
temp_max                    float64
pressure                      int64
humidity                      int64
Clear                         int64
Clouds                        int64
Drizzle                       int64
Fog                           int64
Mist                          int64
Rain                          int64
Smoke                         int64
Snow                          int64
DATES                datetime64[ns]
dtype: object

In [25]:
df_total["DELAY"].describe().mean()

527.3241508795878

In [26]:
df_total["two_minute_delay"] = df_total["DELAY"] < 120
df_total["two_minute_delay"].replace({True:1,False:0},inplace=True)

df_total["five_minute_delay"] = df_total["DELAY"] < 300
df_total["five_minute_delay"].replace({True:1,False:0},inplace=True)

df_total["significant_delay"] = df_total["DELAY"] > df_total["DELAY"].describe().mean()
df_total["significant_delay"].replace({True:1,False:0},inplace=True)

In [27]:
df_total

,TRIPID,DATE,DIRECTION,PLANNEDTIME_TOTAL,ACTUALTIME_TOTAL,DELAY,LINEID,ROUTEID,hours,month,...,Drizzle,Fog,Mist,Rain,Smoke,Snow,DATES,two_minute_delay,five_minute_delay,significant_delay
0,6261203,2018-02-07 09:43:00,0,4735,5265.0,530.0,39A,39A_43,9.0,2,...,0,0,0,0,0,0,2018-02-07 09:00:00,0,0,1
1,6246028,2018-02-07 12:19:00,1,4876,5656.0,780.0,39A,39A_40,12.0,2,...,0,0,0,0,0,0,2018-02-07 12:00:00,0,0,1
2,6251013,2018-02-07 18:30:00,1,4687,5932.0,1245.0,39A,39A_40,18.0,2,...,0,0,0,0,0,0,2018-02-07 18:00:00,0,0,1
3,6274915,2018-02-18 08:19:00,0,3571,3920.0,349.0,39A,39A_43,8.0,2,...,0,0,0,0,0,0,2018-02-18 08:00:00,0,0,0
4,6271772,2018-02-18 10:15:00,0,4163,4091.0,-72.0,39A,39A_43,10.0,2,...,0,0,0,1,0,0,2018-02-18 10:00:00,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,8024541,2018-10-11 09:19:00,0,5190,5557.0,367.0,39A,39A_43,9.0,10,...,0,0,0,0,0,0,2018-10-11 09:00:00,0,0,0
123,8025109,2018-10-11 09:29:00,0,5190,5587.0,397.0,39A,39A_43,9.0,10,...,0,0,0,0,0,0,2018-10-11 09:00:00,0,0,0
124,8020475,2018-10-11 09:51:00,0,5190,5419.0,229.0,39A,39A_43,9.0,10,...,0,0,0,0,0,0,2018-10-11 09:00:00,0,1,0
125,8023269,2018-10-11 18:21:00,0,5121,6079.0,958.0,39A,39A_43,18.0,10,...,0,0,0,0,0,0,2018-10-11 18:00:00,0,0,1


In [28]:
x_train, x_test, y_train, y_test = train_test_split(df_total[features], df_total["significant_delay"], test_size=0.3, random_state=0)

In [29]:
# building the models
# Model 1
logreg = LogisticRegression().fit(x_train[features], y_train)

In [30]:
#logistic regression results
logreg_table = {"Features":features, "Coefficient": logreg.coef_[0]}
print("\nIntercept: \n", logreg.intercept_,"\n")

print(tabulate(logreg_table, headers='keys'))


Intercept: 
 [-0.02469964] 

Features      Coefficient
----------  -------------
DIRECTION     0.413033
pressure      0.000885299
Mist          0.75282
Rain         -0.27369
humidity     -0.0163709


In [34]:
t = 0.5

logreg_predictions_train = (logreg.predict(x_train[features]) >= t) * 1.0

print("==================== Train Data =======================")
print("Overall Performance:")
print("Accuracy: {:2f}".format(metrics.accuracy_score(y_train, logreg_predictions_train)))
print("Precision: {:2f}".format(metrics.precision_score(y_train, logreg_predictions_train)))
print("f1: {:2f}".format(metrics.f1_score(y_train, logreg_predictions_train)))
print("Recall: {:2f}".format(metrics.recall_score(y_train,logreg_predictions_train)))   
print("\nConfusion matrix:")
print("\t\tBus Predicted Late by more than the mean")
print("Details\t\tNo\tYes")
print("Late (No)\t{}\t{}".format(metrics.confusion_matrix(y_train, logreg_predictions_train)[0][0],metrics.confusion_matrix(y_train, logreg_predictions_train)[0][1]))
print("Late (Yes)\t{}\t{}".format(metrics.confusion_matrix(y_train, logreg_predictions_train)[1][0],metrics.confusion_matrix(y_train, logreg_predictions_train)[1][1]))

print("\nClassification report:\n ", metrics.classification_report(y_train, logreg_predictions_train))
print("======================================================")

==================== Train Data =======================
Overall Performance:
Accuracy: 0.534091
Precision: 0.464286
f1: 0.388060
Recall: 0.333333

Confusion matrix:
		Bus Predicted Late by more than the mean
Details		No	Yes
Late (No)	34	15
Late (Yes)	26	13

Classification report:
                precision    recall  f1-score   support

           0       0.57      0.69      0.62        49
           1       0.46      0.33      0.39        39

    accuracy                           0.53        88
   macro avg       0.52      0.51      0.51        88
weighted avg       0.52      0.53      0.52        88



In [35]:
a = logreg_predictions_train

unique, counts = np.unique(a, return_counts=True)

In [36]:
metrics.confusion_matrix(y_train, logreg_predictions_train)

array([[34, 15],
       [26, 13]], dtype=int64)

In [37]:
logreg_predictions_test = (logreg.predict(x_test[features]))# >= t) * 1.0
#df_test["prediction_logreg"] = logreg_predictions_test 

In [38]:
#evaluation metrics (test)
print("\n==================== Test Data ======================")
print("Overall Performance:")
print("Accuracy: {:2f}".format(metrics.accuracy_score(y_test, logreg_predictions_test)))
print("Precision: {:2f}".format(metrics.precision_score(y_test, logreg_predictions_test)))
print("f1: {:2f}".format(metrics.f1_score(y_test, logreg_predictions_test)))
print("Recall: {:2f}".format(metrics.recall_score(y_test, logreg_predictions_test)))   
print("\nConfusion matrix:")
print("\t\tBus Late by more than the mean")
print("Details\t\tNo\tYes")
print("Late (No)\t{}\t{}".format(metrics.confusion_matrix(y_test, logreg_predictions_test)[0][0],metrics.confusion_matrix(y_test, logreg_predictions_test)[0][1]))
print("Late (Yes)\t{}\t{}".format(metrics.confusion_matrix(y_test, logreg_predictions_test)[1][0],metrics.confusion_matrix(y_test, logreg_predictions_test)[1][1]))

print("\nClassification report:\n ", metrics.classification_report(y_test, logreg_predictions_test))
print("======================================================")


==================== Test Data ======================
Overall Performance:
Accuracy: 0.589744
Precision: 0.800000
f1: 0.500000
Recall: 0.363636

Confusion matrix:
		Bus Late by more than the mean
Details		No	Yes
Late (No)	15	2
Late (Yes)	14	8

Classification report:
                precision    recall  f1-score   support

           0       0.52      0.88      0.65        17
           1       0.80      0.36      0.50        22

    accuracy                           0.59        39
   macro avg       0.66      0.62      0.58        39
weighted avg       0.68      0.59      0.57        39



In [39]:
#test for 5 minute delay
x_train, x_test, y_train, y_test = train_test_split(df_total[features], df_total["five_minute_delay"], test_size=0.3, random_state=0)
logreg5minutes = LogisticRegression().fit(x_train[features], y_train)

In [40]:
#logistic regression results
logreg_table5 = {"Features":features, "Coefficient": logreg5minutes.coef_[0]}
print("\nIntercept: \n", logreg5minutes.intercept_,"\n")

print(tabulate(logreg_table5, headers='keys'))


Intercept: 
 [0.00241577] 

Features      Coefficient
----------  -------------
DIRECTION    -0.381979
pressure     -0.000230666
Mist         -0.227667
Rain         -0.108978
humidity     -0.00192296


In [41]:
t = 0.5

logreg_predictions_train5 = (logreg5minutes.predict(x_train[features]) >= t) * 1.0

print("==================== Train Data =======================")
print("Overall Performance:")
print("Accuracy: {:2f}".format(metrics.accuracy_score(y_train, logreg_predictions_train5)))
print("Precision: {:2f}".format(metrics.precision_score(y_train, logreg_predictions_train5)))
print("f1: {:2f}".format(metrics.f1_score(y_train, logreg_predictions_train5)))
print("Recall: {:2f}".format(metrics.recall_score(y_train,logreg_predictions_train5)))   
print("\nConfusion matrix:")
print("\t\tBus Late by less than 5 minutes")
print("Details\t\tNo\tYes")
print("Late (No)\t{}\t{}".format(metrics.confusion_matrix(y_train, logreg_predictions_train5)[0][0],metrics.confusion_matrix(y_train, logreg_predictions_train5)[0][1]))
print("Late (Yes)\t{}\t{}".format(metrics.confusion_matrix(y_train, logreg_predictions_train5)[1][0],metrics.confusion_matrix(y_train, logreg_predictions_train5)[1][1]))

print("\nClassification report:\n ", metrics.classification_report(y_train, logreg_predictions_train5))
print("======================================================")

==================== Train Data =======================
Overall Performance:
Accuracy: 0.636364
Precision: 0.000000
f1: 0.000000
Recall: 0.000000

Confusion matrix:
		Bus Late by less than 5 minutes
Details		No	Yes
Late (No)	56	0
Late (Yes)	32	0

Classification report:
                precision    recall  f1-score   support

           0       0.64      1.00      0.78        56
           1       0.00      0.00      0.00        32

    accuracy                           0.64        88
   macro avg       0.32      0.50      0.39        88
weighted avg       0.40      0.64      0.49        88



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.p

In [42]:
logreg_predictions_test5 = (logreg5minutes.predict(x_test[features]) >= t)

In [43]:
#evaluation metrics (test)
print("\n==================== Test Data ======================")
print("Overall Performance:")
print("Accuracy: {:2f}".format(metrics.accuracy_score(y_test, logreg_predictions_test5)))
print("Precision: {:2f}".format(metrics.precision_score(y_test, logreg_predictions_test5)))
print("f1: {:2f}".format(metrics.f1_score(y_test, logreg_predictions_test5)))
print("Recall: {:2f}".format(metrics.recall_score(y_test, logreg_predictions_test5)))   
print("\nConfusion matrix:")
print("\t\tBus Late by less than 5 minutes")
print("Details\t\tNo\tYes")
print("Late (No)\t{}\t{}".format(metrics.confusion_matrix(y_test, logreg_predictions_test5)[0][0],metrics.confusion_matrix(y_test, logreg_predictions_test5)[0][1]))
print("Late (Yes)\t{}\t{}".format(metrics.confusion_matrix(y_test, logreg_predictions_test5)[1][0],metrics.confusion_matrix(y_test, logreg_predictions_test5)[1][1]))

print("\nClassification report:\n ", metrics.classification_report(y_test, logreg_predictions_test5))
print("======================================================")


==================== Test Data ======================
Overall Performance:
Accuracy: 0.743590
Precision: 0.000000
f1: 0.000000
Recall: 0.000000

Confusion matrix:
		Bus Late by less than 5 minutes
Details		No	Yes
Late (No)	29	0
Late (Yes)	10	0

Classification report:
                precision    recall  f1-score   support

           0       0.74      1.00      0.85        29
           1       0.00      0.00      0.00        10

    accuracy                           0.74        39
   macro avg       0.37      0.50      0.43        39
weighted avg       0.55      0.74      0.63        39



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.p

In [44]:
x_train, x_test, y_train, y_test = train_test_split(df_total[features], df_total["two_minute_delay"], test_size=0.3, random_state=0)

In [45]:
logreg2 = LogisticRegression().fit(x_train[features], y_train)

In [46]:
#logistic regression results
logreg_table2 = {"Features":features, "Coefficient": logreg2.coef_[0]}
print("\nIntercept: \n", logreg2.intercept_,"\n")

print(tabulate(logreg_table2, headers='keys'))


Intercept: 
 [0.00091549] 

Features      Coefficient
----------  -------------
DIRECTION     -0.42451
pressure      -0.00103142
Mist          -0.552953
Rain          -0.058477
humidity       0.00261837


# Random Forests Model

In [47]:
#random forest models
rfc = RandomForestClassifier(n_estimators=100, max_features='auto', oob_score=True, random_state=0)
rfc.fit(x_train[features], y_train)

RandomForestClassifier(oob_score=True, random_state=0)

In [48]:
# build random forests on training set
dtc2 = DecisionTreeClassifier(max_depth=2, random_state=0)
dtc4 = DecisionTreeClassifier(max_depth=4, random_state=0)
dtc10 = DecisionTreeClassifier(max_depth=10, random_state=0)
dtc2.fit(x_train[features], y_train)
dtc4.fit(x_train[features], y_train)
dtc10.fit(x_train[features], y_train)
print("Max depth 2: \n",dtc2)
print("Max depth 4: \n",dtc4)
print("Max depth 10: \n",dtc10)

Max depth 2: 
 DecisionTreeClassifier(max_depth=2, random_state=0)
Max depth 4: 
 DecisionTreeClassifier(max_depth=4, random_state=0)
Max depth 10: 
 DecisionTreeClassifier(max_depth=10, random_state=0)


In [49]:
#obtain graphical representation of decision trees
#NOTE: depending on how graphviz is set up on your computer, this operation might not work, so I have included a copy of 
#the trees as pngs
with open("DecisionTree2.dot", 'w') as f0:
    f1 = export_graphviz(dtc2, out_file=f0, feature_names=x_train[features].columns)
with open("DecisionTree4.dot", 'w') as f1:
    f1 = export_graphviz(dtc4, out_file=f1, feature_names=x_train[features].columns)
with open("DecisionTree10.dot", 'w') as f2:
    f2 = export_graphviz(dtc10, out_file=f2, feature_names=x_train[features].columns)
!dot -Tpng DecisionTree2.dot -o DecisionTree2.png
!dot -Tpng DecisionTree4.dot -o DecisionTree4.png
!dot -Tpng DecisionTree10.dot -o DecisionTree10.png

'dot' is not recognized as an internal or external command,
operable program or batch file.
'dot' is not recognized as an internal or external command,
operable program or batch file.
'dot' is not recognized as an internal or external command,
operable program or batch file.


<img src="DecisionTree2.png">